In [1]:
# @title Install and Setup - Upload Files
!pip install onnx onnxruntime opencv-python insightface gdown tqdm

import datetime
import numpy as np
import os
import os.path as osp
import glob
import cv2
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
import matplotlib.pyplot as plt
import gdown
from PIL import Image
from google.colab import files
from IPython.display import clear_output
from tqdm import tqdm

def upload_files():
    uploaded = files.upload()
    return list(uploaded.keys())

#/root/.insightface/models/buffalo_l.zip
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(640, 640))
# Import the library
import gdown

# Google Drive file ID
file_id = '1krOLgjW2tAPaqV-Bw4YALz0xT5zlb5HF'
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, 'inswapper_128.onnx', quiet=False)

swapper = insightface.model_zoo.get_model('inswapper_128.onnx',download=False,download_zip=False)

os.system("mkdir output")
os.system("mkdir frame")
clear_output()

face_image = upload_files()
image_files = upload_files()

clear_output()

In [58]:
# @title Run FaceSwap
def faceswapdir(face_image,path="."):
  img1 = cv2.imread(face_image)
  lst = os.listdir(path)
  lst.sort()
  img_lst = [cv2.imread(f"{path}/{img}") for img in lst]
  base_face = app.get(img1)[0]
  face_lst =[app.get(face)[0] for face in img_lst]
  out_img = [swapper.get(swap.copy(), face_lst[i], base_face, paste_back=True) for i,swap in enumerate(tqdm(img_lst, desc="Please Wait"))]
  swapped_img1_rgb = [cv2.cvtColor(swapped, cv2.COLOR_BGR2RGB) for swapped in out_img]

  for i,swapped2 in enumerate(swapped_img1_rgb):
    Image.fromarray(np.uint8(swapped2)).save(f"output/{i+1:06}.jpg")


#faceswap(face_image[0],image_files,"/content/frame2")


In [ ]:
# @title Only face upload
face_image = upload_files()
print('insightface', insightface.__version__)
print('numpy', np.__version__)

# Check that InsightFace version is at least 0.7
assert float('.'.join(insightface.__version__.split('.')[:2])) >= float('0.7')

In [ ]:
# @title Only images upload
image_files = upload_files()


In [ ]:
!ffmpeg -hide_banner -loglevel error -i sample.mov -vcodec h264 -acodec aac sample.mp4

In [ ]:
!ffmpeg -hide_banner -loglevel error  -i sample.mp4 -q:a 0 -map a sample.mp3

In [ ]:
!ffmpeg -hide_banner -loglevel error  -hwaccel auto -i sample.mp4 -q:v 18 -pix_fmt rgb24 -vf fps=30  /content/frame/%06d.jpg

In [ ]:
faceswapdir(face_image[0],"/content/frame")

In [ ]:
!ffmpeg -hide_banner -loglevel error  -hwaccel auto -r 30 -i /content/output/%06d.jpg -vcodec libx264 -crf 18  -pix_fmt yuv420p   video.mp4

In [ ]:
!ffmpeg -hide_banner -loglevel error  -i video.mp4 -i audio.wav -c:v copy -c:a aac output.mp4

In [52]:
# @title Remove all files  from frame/output folder
!rm -r /content/frame
!rm -r /content/output
!mkdir /content/output
!mkdir /content/frame